# Import Libraries

In [ ]:
import requests
import pandas as pd

# Read files and merge files

In [ ]:
user = "chencba"
repo = "EnergyDashboards"

# Get all the csv file names in the github repository
# https://api.github.com/repos/[USER]/[REPO]/git/trees/[BRANCH]?recursive=1
url = "https://api.github.com/repos/{}/{}/git/trees/main?recursive=1".format(user, repo)

r = requests.get(url)
res = r.json()

file_list = []
for file in res["tree"]:
  file_list.append(file["path"])

In [ ]:
# Show all the file names
file_list

['AEM-export-70653-UCD_Belfield_ Belfield Main Elec & Gas_Electricity_Total Main Campus Elec Consumption_Belfield Electricity.csv',
 'AEM-export-70654-UCD_Belfield_Agriculture & Food Science_Agriculture & Food Science Building Electricity.csv',
 'AEM-export-70655-UCD_Belfield_Computer Science_Computer Science Electricity.csv',
 'AEM-export-70656-UCD_Belfield_Computer Centre_Electricity_Computer Centre.csv',
 'AEM-export-70657-UCD_Belfield_Confucius_Confucius Main Electricity.csv',
 'AEM-export-70658-UCD_Belfield_Health Sciences_Health Science Total Electricity (Gross)_Health Science Total Electricity.csv',
 'AEM-export-70659-UCD_Belfield_Library (James Joyce)_Electricity_James Joyce Library Building_James Joyce Library Building Net Meter.csv',
 'AEM-export-70660-UCD_Belfield_Newman Building (Arts)_Electricity_Newman Building (Arts) Electricity.csv',
 'AEM-export-70661-UCD_Belfield_Nova UCD_Nova Total Electricity.csv',
 "AEM-export-70662-UCD_Belfield_O'Reilly Hall_O'Reilly Hall Electric

In [ ]:
# Total number of files
len(file_list)

23

In [ ]:
# Remove the two main campus's electricity files
file_list.remove('AEM-export-70653-UCD_Belfield_ Belfield Main Elec & Gas_Electricity_Total Main Campus Elec Consumption_Belfield Electricity.csv')
file_list.remove('AEM-export-70668-UCD_Blackrock_ Blackrock Main Electricity.csv')

In [ ]:
# Now we have 21 files including the 3 weather files
len(file_list)

21

In [ ]:
# Read all the csv files (except the weather files) and store them in a list
base_url = "https://raw.githubusercontent.com/chencba/EnergyDashboards/main/"

files = []
for each in file_list[:-3]:
  loc = each.split("_")[-1][:-4]
  s = each.replace(" ", "%20")
  s = s.replace("&", "%26")
  data = pd.read_csv(base_url + s, header = 1, usecols = [0, 1])
  data.rename(columns={data.columns[0]:"timestamp", data.columns[1]:"electricity_consumption"}, inplace = True)
  data["timestamp"] = pd.to_datetime(data["timestamp"])
  data = data[(data["timestamp"] >= "2015-01-01") & (data["timestamp"] < "2022-07-20")]
  data = data.resample("1H", on = "timestamp").sum()  # change the frequency from 15m to 1H
  data["building"] = loc  # get building names
  files.append(data)

In [ ]:
# Merge all the files
df = pd.concat(files)

In [ ]:
df

,electricity_consumption,building
timestamp,,
2015-01-01 00:00:00,109.264999,Agriculture & Food Science Building Electricity
2015-01-01 01:00:00,111.284996,Agriculture & Food Science Building Electricity
2015-01-01 02:00:00,106.947498,Agriculture & Food Science Building Electricity
2015-01-01 03:00:00,115.107498,Agriculture & Food Science Building Electricity
2015-01-01 04:00:00,107.972498,Agriculture & Food Science Building Electricity
...,...,...
2022-07-19 19:00:00,22.540000,MH Total Electricity
2022-07-19 20:00:00,12.097500,MH Total Electricity
2022-07-19 21:00:00,13.032500,MH Total Electricity


In [ ]:
# View all the buildings
df["building"].unique()

array(['Agriculture & Food Science Building Electricity',
       'Computer Science Electricity', 'Computer Centre',
       'Confucius Main Electricity', 'Health Science Total Electricity',
       'James Joyce Library Building Net Meter',
       'Newman Building (Arts) Electricity', 'Nova Total Electricity',
       "O'Reilly Hall Electricity", 'Quinn School Electricity',
       'Sports Centre Electricity', 'Student Centre Electricity',
       'Sutherland School of Law Electricity',
       'Blackrock Library Electricity',
       'Blackrock Restaurant Electricity', 'East & West Hall Electricity',
       'Proby House Electricity', 'MH Total Electricity'], dtype=object)

In [ ]:
# Check how many different buidlings are included
len(df["building"].unique())

18

In [ ]:
# Load the files for the two campuses and save the info into a separate file - df_campus
belfield = pd.read_csv('https://raw.githubusercontent.com/chencba/EnergyDashboards/main/AEM-export-70653-UCD_Belfield_%20Belfield%20Main%20Elec%20%26%20Gas_Electricity_Total%20Main%20Campus%20Elec%20Consumption_Belfield%20Electricity.csv', header = 1, usecols = [0, 1])
blackrock = pd.read_csv('https://raw.githubusercontent.com/chencba/EnergyDashboards/main/AEM-export-70668-UCD_Blackrock_%20Blackrock%20Main%20Electricity.csv', header = 1, usecols = [0, 1])
belfield.rename(columns={belfield.columns[0]:"timestamp", belfield.columns[1]:"electricity_consumption"}, inplace = True)
blackrock.rename(columns={blackrock.columns[0]:"timestamp", blackrock.columns[1]:"electricity_consumption"}, inplace = True)
belfield["timestamp"] = pd.to_datetime(belfield["timestamp"])
blackrock["timestamp"] = pd.to_datetime(blackrock["timestamp"])
belfield = belfield[(belfield["timestamp"] >= "2015-01-01") & (belfield["timestamp"] < "2022-07-20")]
blackrock = blackrock[(blackrock["timestamp"] >= "2015-01-01") & (blackrock["timestamp"] < "2022-07-20")]
belfield = belfield.resample("1H", on = "timestamp").sum()
blackrock = blackrock.resample("1H", on = "timestamp").sum()
belfield["campus"] = "Belfield"
blackrock["campus"] = "Blackrock"
df_campus = pd.concat([belfield, blackrock])

In [ ]:
df_campus

,electricity_consumption,campus
timestamp,,
2015-01-01 00:00:00,2402.000000,Belfield
2015-01-01 01:00:00,2377.000000,Belfield
2015-01-01 02:00:00,2371.000000,Belfield
2015-01-01 03:00:00,2387.000000,Belfield
2015-01-01 04:00:00,2460.000000,Belfield
...,...,...
2022-07-19 19:00:00,100.442499,Blackrock
2022-07-19 20:00:00,93.574997,Blackrock
2022-07-19 21:00:00,88.374996,Blackrock


# Add weather data

In [ ]:
lux = pd.read_csv("https://raw.githubusercontent.com/chencba/EnergyDashboards/main/AEM-export-70674-UCD_Weather%20Data_Light%20Lux%20(Imported).csv", header = 1, usecols = [0, 1])
temperature = pd.read_csv("https://raw.githubusercontent.com/chencba/EnergyDashboards/main/AEM-export-70675-UCD_Weather%20Data_Outside%20Air%20Temperature.csv", header = 1, usecols = [0, 1])
humidity = pd.read_csv("https://raw.githubusercontent.com/chencba/EnergyDashboards/main/AEM-export-70676-UCD_Weather%20Data_Relative%20Humidity.csv", header = 1, usecols = [0, 1])

In [ ]:
lux.rename(columns={lux.columns[0]:"timestamp", lux.columns[1]:"light_lux"}, inplace = True)
temperature.rename(columns={temperature.columns[0]:"timestamp", temperature.columns[1]:"outside_temperature"}, inplace = True)
humidity.rename(columns={humidity.columns[0]:"timestamp", humidity.columns[1]:"relative_humidity"}, inplace = True)

In [ ]:
lux["timestamp"] = pd.to_datetime(lux["timestamp"])
temperature["timestamp"] = pd.to_datetime(temperature["timestamp"])
humidity["timestamp"] = pd.to_datetime(humidity["timestamp"])

In [ ]:
lux = lux.resample("1H", on = "timestamp").mean()
temperature = temperature.resample("1H", on = "timestamp").mean()
humidity = humidity.resample("1H", on = "timestamp").mean()

In [ ]:
df = df.merge(lux, how = "left", on = "timestamp")
df = df.merge(temperature, how = "left", on = "timestamp")
df = df.merge(humidity, how = "left", on = "timestamp")

In [ ]:
df_campus = df_campus.merge(lux, how = "left", on = "timestamp")
df_campus = df_campus.merge(temperature, how = "left", on = "timestamp")
df_campus = df_campus.merge(humidity, how = "left", on = "timestamp")

In [ ]:
df[:5]

,electricity_consumption,building,light_lux,outside_temperature,relative_humidity
timestamp,,,,,
2015-01-01 00:00:00,109.264999,Agriculture & Food Science Building Electricity,122.896355,11.715427,30.404237
2015-01-01 01:00:00,111.284996,Agriculture & Food Science Building Electricity,124.211668,12.003126,45.249629
2015-01-01 02:00:00,106.947498,Agriculture & Food Science Building Electricity,128.879084,12.578514,54.757489
2015-01-01 03:00:00,115.107498,Agriculture & Food Science Building Electricity,127.787912,12.857532,56.241366
2015-01-01 04:00:00,107.972498,Agriculture & Food Science Building Electricity,129.823078,13.270016,54.726522


In [ ]:
df.isnull().sum()

electricity_consumption        0
building                       0
light_lux                  17166
outside_temperature        15239
relative_humidity          15239
dtype: int64

In [ ]:
df_campus[:5]

,electricity_consumption,campus,light_lux,outside_temperature,relative_humidity
timestamp,,,,,
2015-01-01 00:00:00,2402.0,Belfield,122.896355,11.715427,30.404237
2015-01-01 01:00:00,2377.0,Belfield,124.211668,12.003126,45.249629
2015-01-01 02:00:00,2371.0,Belfield,128.879084,12.578514,54.757489
2015-01-01 03:00:00,2387.0,Belfield,127.787912,12.857532,56.241366
2015-01-01 04:00:00,2460.0,Belfield,129.823078,13.270016,54.726522


In [ ]:
"""
# Add campus information
blackrock = [' Blackrock Main Electricity', 'Blackrock Library Electricity',
       'Blackrock Restaurant Electricity', 'East & West Hall Electricity',
       'Proby House Electricity', 'MH Total Electricity']
df["campus"] = ["Belfield" if i not in blackrock else "Blackrock" for i in df["building"]]
"""

'\n# Add campus information\nblackrock = [\' Blackrock Main Electricity\', \'Blackrock Library Electricity\',\n       \'Blackrock Restaurant Electricity\', \'East & West Hall Electricity\',\n       \'Proby House Electricity\', \'MH Total Electricity\']\ndf["campus"] = ["Belfield" if i not in blackrock else "Blackrock" for i in df["building"]]\n'

In [ ]:
"""
# Add coordinates
coord = {'Belfield Electricity': [53.30736, -6.22391],
       'Agriculture & Food Science Building Electricity': [53.30732, -6.22452],
       'Computer Science Electricity': [53.30932, -6.22390], 'Computer Centre': [53.30952, -6.22484],
       'Confucius Main Electricity': [53.30587, -6.21777], 'Health Science Total Electricity': [53.30930, -6.33596],
       'James Joyce Library Building Net Meter': [53.30676, -6.22315],
       'Newman Building (Arts) Electricity': [53.30601, -6.22166], 'Nova Total Electricity': [53.30455, -6.21189],
       "O'Reilly Hall Electricity": [53.30856, -6.22118], 'Quinn School Electricity': [53.30465, -6.21999],
       'Sports Centre Electricity': [53.30827, -6.22952], 'Total Sports Pitch Lighting': [53.30680, -6.22911],
       'Student Centre Electricity': [53.30829, -6.22650],
       'Sutherland School of Law Electricity': [53.30441, -6.21882],
       ' Blackrock Main Electricity': [53.29560, -6.18494], 'Blackrock Library Electricity': [53.29524, -6.18416],
       'Blackrock Restaurant Electricity': [53.29576, -6.18689], 'East & West Hall Electricity': [53.29578, -6.18634],
       'Proby House Electricity': [53.29460, -6.18486], 'MH Total Electricity': [53.29489, -6.18443]}
df["coord"] = df["building"].map(coord)
df["longitude"] = df["coord"].apply(lambda x: x[0])
df["latitude"] = df["coord"].apply(lambda x: x[1])
df.drop("coord", axis = 1, inplace = True)
"""

'\n# Add coordinates\ncoord = {\'Belfield Electricity\': [53.30736, -6.22391],\n       \'Agriculture & Food Science Building Electricity\': [53.30732, -6.22452],\n       \'Computer Science Electricity\': [53.30932, -6.22390], \'Computer Centre\': [53.30952, -6.22484],\n       \'Confucius Main Electricity\': [53.30587, -6.21777], \'Health Science Total Electricity\': [53.30930, -6.33596],\n       \'James Joyce Library Building Net Meter\': [53.30676, -6.22315],\n       \'Newman Building (Arts) Electricity\': [53.30601, -6.22166], \'Nova Total Electricity\': [53.30455, -6.21189],\n       "O\'Reilly Hall Electricity": [53.30856, -6.22118], \'Quinn School Electricity\': [53.30465, -6.21999],\n       \'Sports Centre Electricity\': [53.30827, -6.22952], \'Total Sports Pitch Lighting\': [53.30680, -6.22911],\n       \'Student Centre Electricity\': [53.30829, -6.22650],\n       \'Sutherland School of Law Electricity\': [53.30441, -6.21882],\n       \' Blackrock Main Electricity\': [53.29560, -

# Add building size info

In [ ]:
area = {'Agriculture & Food Science Building Electricity': 10529,
     'Computer Science Electricity': 2855, 'Computer Centre': 893,
     'Confucius Main Electricity': 1935, 'Health Science Total Electricity': 17804,
     'James Joyce Library Building Net Meter': 21182,
     'Newman Building (Arts) Electricity': 25288, 'Nova Total Electricity': 5321,
     "O'Reilly Hall Electricity": 5617, 'Quinn School Electricity': 9883,
     'Sports Centre Electricity': 6488,
     'Student Centre Electricity': 14858,
     'Sutherland School of Law Electricity': 5974,
     'Blackrock Library Electricity': 931,
     'Blackrock Restaurant Electricity': 1234, 'East & West Hall Electricity': 3292,
     'Proby House Electricity': 3764, 'MH Total Electricity': 1614}

In [ ]:
df["area"] = df["building"].map(area)

In [ ]:
df[:5]

,electricity_consumption,building,light_lux,outside_temperature,relative_humidity,area
timestamp,,,,,,
2015-01-01 00:00:00,109.264999,Agriculture & Food Science Building Electricity,122.896355,11.715427,30.404237,10529
2015-01-01 01:00:00,111.284996,Agriculture & Food Science Building Electricity,124.211668,12.003126,45.249629,10529
2015-01-01 02:00:00,106.947498,Agriculture & Food Science Building Electricity,128.879084,12.578514,54.757489,10529
2015-01-01 03:00:00,115.107498,Agriculture & Food Science Building Electricity,127.787912,12.857532,56.241366,10529
2015-01-01 04:00:00,107.972498,Agriculture & Food Science Building Electricity,129.823078,13.270016,54.726522,10529


# Add CO2e emission and cost information

In [ ]:
# CO2e intensity in Ireland in 2022: 0.525 kg CO2e per kWh, unit price of electricity: 0.2812 EURO per kWh 
df["CO2e_emission"] = df["electricity_consumption"] * 0.525
df["cost"] = df["electricity_consumption"] * 0.1485

In [ ]:
df_campus["CO2e_emission"] = df_campus["electricity_consumption"] * 0.525
df_campus["cost"] = df_campus["electricity_consumption"] * 0.1485

In [ ]:
df[:5]

,electricity_consumption,building,light_lux,outside_temperature,relative_humidity,area,CO2e_emission,cost
timestamp,,,,,,,,
2015-01-01 00:00:00,109.264999,Agriculture & Food Science Building Electricity,122.896355,11.715427,30.404237,10529,57.364125,16.225852
2015-01-01 01:00:00,111.284996,Agriculture & Food Science Building Electricity,124.211668,12.003126,45.249629,10529,58.424623,16.525822
2015-01-01 02:00:00,106.947498,Agriculture & Food Science Building Electricity,128.879084,12.578514,54.757489,10529,56.147437,15.881704
2015-01-01 03:00:00,115.107498,Agriculture & Food Science Building Electricity,127.787912,12.857532,56.241366,10529,60.431437,17.093463
2015-01-01 04:00:00,107.972498,Agriculture & Food Science Building Electricity,129.823078,13.270016,54.726522,10529,56.685561,16.033916


In [ ]:
df_campus[:5]

,electricity_consumption,campus,light_lux,outside_temperature,relative_humidity,CO2e_emission,cost
timestamp,,,,,,,
2015-01-01 00:00:00,2402.0,Belfield,122.896355,11.715427,30.404237,1261.050,356.6970
2015-01-01 01:00:00,2377.0,Belfield,124.211668,12.003126,45.249629,1247.925,352.9845
2015-01-01 02:00:00,2371.0,Belfield,128.879084,12.578514,54.757489,1244.775,352.0935
2015-01-01 03:00:00,2387.0,Belfield,127.787912,12.857532,56.241366,1253.175,354.4695
2015-01-01 04:00:00,2460.0,Belfield,129.823078,13.270016,54.726522,1291.500,365.3100


# Add normalized electricity_consumption, normalized CO2 emission and normalized cost information

In [ ]:
df["normalized_electricity_consumption"] = df["electricity_consumption"] / df["area"]
df["normalized_cost"] = df["cost"] / df["area"]
df["normalized_CO2e"] = df["CO2e_emission"] / df["area"]

In [ ]:
df[:5]

,electricity_consumption,building,light_lux,outside_temperature,relative_humidity,area,CO2e_emission,cost,normalized_electricity_consumption,normalized_cost,normalized_CO2e
timestamp,,,,,,,,,,,
2015-01-01 00:00:00,109.264999,Agriculture & Food Science Building Electricity,122.896355,11.715427,30.404237,10529,57.364125,16.225852,0.010378,0.001541,0.005448
2015-01-01 01:00:00,111.284996,Agriculture & Food Science Building Electricity,124.211668,12.003126,45.249629,10529,58.424623,16.525822,0.010569,0.001570,0.005549
2015-01-01 02:00:00,106.947498,Agriculture & Food Science Building Electricity,128.879084,12.578514,54.757489,10529,56.147437,15.881704,0.010157,0.001508,0.005333
2015-01-01 03:00:00,115.107498,Agriculture & Food Science Building Electricity,127.787912,12.857532,56.241366,10529,60.431437,17.093463,0.010932,0.001623,0.005740
2015-01-01 04:00:00,107.972498,Agriculture & Food Science Building Electricity,129.823078,13.270016,54.726522,10529,56.685561,16.033916,0.010255,0.001523,0.005384


# Save the final datasets - buildings and campus

In [ ]:
df.to_csv("UCD_electricity_buildings.csv")
df_campus.to_csv("UCD_electricity_campus.csv")